In [1]:
import pandas as pd
import pickle
import time
import multiprocessing
from multiprocessing import Process
import math
import json

In [2]:
df=pd.read_json("jsons/Major_dataset.json")

In [3]:
aut_list=pickle.load(open("pickles/aut_list.pkl","rb"))

In [4]:
alist=[]
paper_dict={key: list(alist) for key in aut_list}

In [5]:
def handle_authors(old_author):
    return(str(old_author)[1:-1])

In [6]:
df['author'] = df['author'].apply(handle_authors)

In [7]:
df=df.reset_index()
df.head()

,index,abstract,author,id,title,venue,year
0,1,In system operations the term rollback is ofte...,Mark Burgess,01705f09-d395-4a0f-a89f-8d18382c81cc,On system rollback and totalized fields: An al...,The Journal of Logic and Algebraic Programming,2011
1,10,The paper describes user behavior as a result ...,Ilya V. Osipov,04b38d22-d10c-4b84-8a9d-bd610400aa4e,Monetization as a Motivator for the Freemium E...,arXiv: Computers and Society,2015
2,100,Electrification is foremost actor in supersedi...,Binod Vaidya,2009175e-2ba9-4a66-97eb-b0da75122f2b,Security and Privacy-Preserving Mechanism for ...,ad hoc networks,2014
3,1000,This contribution proposes a model for argumen...,Eva Onaindia,e81bbc24-49b0-46bf-a43c-76da491be937,Multiagent argumentation for cooperative plann...,adaptive agents and multi agents systems,2011
4,10000,In the paper a genetic algorithm approach to f...,Andrea Cardoni,4b7cb203-3454-4b61-9273-b74e964c449f,A Genetic Algorithm Approach for Collaborative...,working conference on virtual enterprises,2012


In [8]:
for i in range(df.shape[0]):
    if i%100000==0:
        print(i//100000,end=' ')
    try:
        paper_dict[df.iloc[i].author].append(df.iloc[i].id)
        #print(len(paper_dict['Rabab K. Ward']),len(paper_dict['Radko Mesiar']))
    except:
        print("",end='')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 

In [56]:
#pickle.dump(paper_dict,open("pickles/paper_dict.pkl","wb"))
paper_dict=pickle.load(open("pickles/paper_dict.pkl","rb"))

In [10]:
papers_list=list(df.id.unique())

In [57]:
#pickle.dump(papers_list,open("pickles/papers_list.pkl","wb"))
papers_list=pickle.load(open("pickles/papers_list.pkl","rb"))

In [12]:
df1=pd.read_json("jsons/Major_dataset_ref.json")

In [13]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5084293 entries, 1 to 999999
Data columns (total 7 columns):
abstract      object
author        object
id            object
references    object
title         object
venue         object
year          int64
dtypes: int64(1), object(6)
memory usage: 310.3+ MB


In [58]:
paper_citation={key:None  for key in papers_list}

In [59]:
df1['author'] = df1['author'].apply(handle_authors)

NameError: name 'df1' is not defined

In [16]:
for i in range(df1.shape[0]):
    if i%100000==0:
        print(i//100000,end=' ')
    paper_citation[df1.iloc[i].id]=len(df1.iloc[i].references)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 

In [60]:
#pickle.dump(paper_citation,open("pickles/paper_citation.pkl","wb"))
paper_citation=pickle.load(open("pickles/paper_citation.pkl","rb"))

In [61]:
citation_dict={key:[] for key in aut_list}

In [19]:
count=0
for aut in paper_dict:
    if count%1000==0:
        print(count//1000,end='')
    for paper in paper_dict[aut]:
        citation_dict[aut].append(paper_citation[paper])
    count+=1

012345678910111213141516171819202122232425262728293031323334353637383940414243444546474849

In [62]:
#pickle.dump(citation_dict,open("pickles/citation_dict.pkl","wb"))
citation_dict=pickle.load(open("pickles/citation_dict.pkl","rb"))

In [63]:
h_index={key:None for key in aut_list}

In [64]:
i=0
for aut in citation_dict:
    if i%1000==0:
        print(i//1000,end='')
    i+=1
    citation_dict[aut].sort(reverse=True)
    count=1
    if len(citation_dict[aut])>0:
        if (citation_dict[aut][0]==0):
            h_index[aut]=0
            continue
    for citation in citation_dict[aut]:
        if citation>=count:
            count+=1
        else:
            break
    h_index[aut]=count
    

012345678910111213141516171819202122232425262728293031323334353637383940414243444546474849

In [4]:
coauthor_list=json.load(open("jsons/co_author_list.json","rb"))

In [66]:
citation_score=[0]*375303

In [67]:
count=0
for coauthor in coauthor_list:
    if count%10000==0:
        print(count//10000,end='')
    ci=citation_dict[coauthor[0]]
    cj=citation_dict[coauthor[1]]
    hi=h_index[coauthor[0]]
    hj=h_index[coauthor[1]]
    score=0
    for i in range(min(hi,hj)-1):
        score+=1/math.log2(abs(ci[i]-cj[i])+2)
    citation_score[count]=score
    count+=1

012345678910111213141516171819202122232425262728293031323334353637

In [68]:
#pickle.dump(citation_score,open("pickles/citation_score.pkl","wb"))
citation_score=pickle.load(open("pickles/citation_score.pkl","rb"))

In [69]:
Coauthor_citation_score=[]
for i in range(375303):
    Coauthor_citation_score.append((coauthor_list[i],citation_score[i]))
pickle.dump(Coauthor_citation_score,open("pickles/Coauthor_citation_score.pkl","wb")) 

In [3]:
# aut_list=[]
# count=0
# for coauthor in coauthor_list:
#     if count%10000==0:
#         print(count//10000,end='')
#     if coauthor[0] not in aut_list:
#         aut_list.append(coauthor[0])
#     if coauthor[1] not in aut_list:
#         aut_list.append(coauthor[1])
#     count+=1

012345678910111213141516171819202122232425262728293031323334353637

In [5]:
# pickle.dump(aut_list,open("pickles/aut_list.pkl","wb"))

In [2]:
Coauthor_citation_score=pickle.load(open("pickles/Coauthor_citation_score.pkl","rb")) 

In [5]:
coauthor_list_new=[]
for coauthor in coauthor_list:
    coauthor_list_new.append((coauthor[0],coauthor[1]))

In [18]:
Coauthor_citation_score_list=[]
for coauthor in Coauthor_citation_score:
    Coauthor_citation_score_list.append(coauthor[1])

In [28]:
m=max(Coauthor_citation_score_list)
norm = [float(i)/m for i in Coauthor_citation_score_list]

In [29]:
print(len(Coauthor_citation_score_list),len(norm))

375303 375303


In [30]:
Coauthor_citation_score_dict=dict(zip(coauthor_list_new,norm))

In [31]:
pickle.dump(Coauthor_citation_score_dict,open("pickles/Coauthor_citation_score_dict.pkl","wb"))

In [32]:
Coauthor_citation_score_dict

{('Romit Roy Choudhury', 'Justin Manweiler'): 0.08836881717649965,
 ('Xuan Bao', 'Justin Manweiler'): 0.12290502466441387,
 ('Kamesh Munagala', 'Justin Manweiler'): 0.11623359545114968,
 ('Souvik Sen', 'Justin Manweiler'): 0.08298227293880195,
 ('Paramvir Bahl', 'Justin Manweiler'): 0.12744224073907262,
 ('Landon P. Cox', 'Justin Manweiler'): 0.1572813588167298,
 ('Ming Zhang', 'Justin Manweiler'): 0.06190098969523361,
 ('Sharad Agarwal', 'Justin Manweiler'): 0.1196453415495379,
 ('Srihari Nelakuditi', 'Justin Manweiler'): 0.1417765877411845,
 ('Hanmin Jung', 'Sung-Pil Choi'): 0.11772434549892696,
 ('Sa-Kwang Song', 'Sung-Pil Choi'): 0.09799637745814506,
 ('Young-Hyun Jun', 'Sung-Pil Choi'): 0.0474247827856357,
 ('Sung-Hyon Myaeng', 'Sung-Pil Choi'): 0.06759772651283583,
 ('Yuchul Jung', 'Sung-Pil Choi'): 0.11779004382467938,
 ('Miguel Rebollo', 'Adriana Giret'): 0.10768407525010652,
 ('Pablo Noriega', 'Adriana Giret'): 0.20349789800392873,
 ('Vicente Julián', 'Adriana Giret'): 0.0,
 (